In [ ]:
'''
    @Author: King
    @Date: 2019.03.18
    @Purpose: 文本分类实战（五）—— Bi-LSTM + Attention模型
    @Introduction:  下面介绍
    @Link : https://github.com/jiangxinyang227/textClassifier/tree/master/Bi-LSTM%2BAttention
    @Reference : https://www.cnblogs.com/jiangxinyang/p/10208227.html
'''

## 数据集介绍

数据集为IMDB 电影影评，总共有三个数据文件，在/data/rawData目录下，包括unlabeledTrainData.tsv，labeledTrainData.tsv，testData.tsv。在进行文本分类时需要有标签的数据（labeledTrainData），数据预处理如文本分类实战（一）—— word2vec预训练词向量中一样，预处理后的文件为/data/preprocess/labeledTrain.csv。


##  Bi-LSTM + Attention 模型介绍
　  
Bi-LSTM + Attention模型来源于论文Attention-Based Bidirectional Long Short-Term Memory Networks for Relation Classification。关于Attention的介绍见这篇。

Bi-LSTM + Attention 就是在Bi-LSTM的模型上加入Attention层，在Bi-LSTM中我们会用最后一个时序的输出向量 作为特征向量，然后进行softmax分类。Attention是先计算每个时序的权重，然后将所有时序 的向量进行加权和作为特征向量，然后进行softmax分类。在实验中，加上Attention确实对结果有所提升。其模型结构如下图：
  
![avatar](../images/BI-LSTM+Attention.png)

In [1]:
import os
import csv
import time
import datetime
import random
import json

import warnings
from collections import Counter
from math import sqrt

import gensim
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
warnings.filterwarnings("ignore")

d:\progrom\python\python\python3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
d:\progrom\python\python\python3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# 配置参数

class TrainingConfig(object):
    epoches = 4
    evaluateEvery = 100
    checkpointEvery = 100
    learningRate = 0.001
    
class ModelConfig(object):
    embeddingSize = 200
    
    hiddenSizes = [256, 128]  # LSTM结构的神经元个数
    
    dropoutKeepProb = 0.5   
    l2RegLambda = 0.0
    
class Config(object):
    sequenceLength = 200  # 取了所有序列长度的均值
    batchSize = 128
    
    dataSource = "../data/preProcess/labeledTrain.csv"
    
    stopWordSource = "../data/english"
    
    numClasses = 2
    
    rate = 0.8  # 训练集的比例
    
    training = TrainingConfig()
    
    model = ModelConfig()

    
# 实例化配置参数对象
config = Config()

## 生成训练数据

　　1）将数据加载进来，将句子分割成词表示，并去除低频词和停用词。

　　2）将词映射成索引表示，构建词汇-索引映射表，并保存成json的数据格式，之后做inference时可以用到。（注意，有的词可能不在word2vec的预训练词向量中，这种词直接用UNK表示）

　　3）从预训练的词向量模型中读取出词向量，作为初始化值输入到模型中。

　　4）将数据集分割成训练集和测试集

In [3]:
# 数据预处理的类，生成训练集和测试集

class Dataset(object):
    def __init__(self, config):
        self._dataSource = config.dataSource          # 数据文件路径
        self._stopWordSource = config.stopWordSource  # 分词文件路径
        
        self._sequenceLength = config.sequenceLength  # 每条输入的序列处理为定长
        self._embeddingSize = config.model.embeddingSize
        self._batchSize = config.batchSize
        self._rate = config.rate
        
        self._stopWordDict = {}
        
        self.trainReviews = []
        self.trainLabels = []
        
        self.evalReviews = []
        self.evalLabels = []
        
        self.wordEmbedding =None
        
        self._wordToIndex = {}
        self._indexToWord = {}
        
    def _readData(self, filePath):
        '''
        从csv文件中读取数据集
        
        :param filePath:   str   数据集文件路径
        :return:
            reviews:   list    数据集
            labels：   list    标签集
        '''
        df = pd.read_csv(filePath)
        labels = df["sentiment"].tolist()
        review = df["review"].tolist()
        reviews = [line.strip().split() for line in review]

        return reviews, labels

    def _reviewProcess(self, review, sequenceLength, wordToIndex):
        '''
        将数据集中的每条评论用index表示
        wordToIndex中“pad”对应的index为0
        
        :param review:   
        :param sequenceLength:  
        :return:
            reviewVec:   list    
        '''
        
        reviewVec = np.zeros((sequenceLength))
        sequenceLen = sequenceLength
        
        # 判断当前的序列是否小于定义的固定序列长度
        if len(review) < sequenceLength:
            sequenceLen = len(review)
            
        for i in range(sequenceLen):
            if review[i] in wordToIndex:
                reviewVec[i] = wordToIndex[review[i]]
            else:
                reviewVec[i] = wordToIndex["UNK"]

        return reviewVec

    def _genTrainEvalData(self, x, y, rate):
        '''
        生成训练集和验证集
        
        :param x:     list     数据
        :param y:     list     数据标签
        :param rate:  float    分割比例
        :return:
            trainReviews:   list    
            trainLabels:   list    
            evalReviews:   list    
            evalLabels:   list    
        '''
        reviews = []
        labels = []
        
        # 遍历所有的文本，将文本中的词转换成index表示
        for i in range(len(x)):
            reviewVec = self._reviewProcess(x[i], self._sequenceLength, self._wordToIndex)
            reviews.append(reviewVec)
            
            labels.append([y[i]])
            
        trainIndex = int(len(x) * rate)
        
        trainReviews = np.asarray(reviews[:trainIndex], dtype="int64")
        trainLabels = np.array(labels[:trainIndex], dtype="float32")
        
        evalReviews = np.asarray(reviews[trainIndex:], dtype="int64")
        evalLabels = np.array(labels[trainIndex:], dtype="float32")

        return trainReviews, trainLabels, evalReviews, evalLabels
        
    def _genVocabulary(self, reviews):
        '''
        生成词向量和词汇-索引映射字典，可以用全数据集
        
        :param reviews:     list     数据
        :return:
            
        '''
        
        allWords = [word for review in reviews for word in review]
        
        # 去掉停用词
        subWords = [word for word in allWords if word not in self.stopWordDict]
        
        wordCount = Counter(subWords)  # 统计词频
        sortWordCount = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
        
        # 去除低频词
        words = [item[0] for item in sortWordCount if item[1] >= 5]
        
        vocab, wordEmbedding = self._getWordEmbedding(words)
        self.wordEmbedding = wordEmbedding
        
        self._wordToIndex = dict(zip(vocab, list(range(len(vocab)))))
        self._indexToWord = dict(zip(list(range(len(vocab))), vocab))
        
        # 将词汇-索引映射表保存为json数据，之后做inference时直接加载来处理数据
        with open("../data/wordJson/wordToIndex.json", "w", encoding="utf-8") as f:
            json.dump(self._wordToIndex, f)
        
        with open("../data/wordJson/indexToWord.json", "w", encoding="utf-8") as f:
            json.dump(self._indexToWord, f)
            
    def _getWordEmbedding(self, words):
        '''
        按照我们的数据集中的单词取出预训练好的word2vec中的词向量
        
        :param words:     list     数据
        :return:
             vocab                       list            词列表
             np.array(wordEmbedding)     numpy array     词向量
            
        '''
        
        wordVec = gensim.models.KeyedVectors.load_word2vec_format("../word2vec/word2Vec.bin", binary=True)
        vocab = []
        wordEmbedding = []
        
        # 添加 "pad" 和 "UNK", 
        vocab.append("pad")
        vocab.append("UNK")
        wordEmbedding.append(np.zeros(self._embeddingSize))
        wordEmbedding.append(np.zeros(self._embeddingSize))
        
        for word in words:
            try:
                vector = wordVec.wv[word]
                vocab.append(word)
                wordEmbedding.append(vector)
            except:
                print(word + "不存在于词向量中")
                
        return vocab, np.array(wordEmbedding)
    
    def _readStopWord(self, stopWordPath):
        '''
        读取停用词
        
        :param stopWordPath:     str     停用词文件路径
        :return: 
            
        '''

        with open(stopWordPath, "r") as f:
            stopWords = f.read()
            stopWordList = stopWords.splitlines()
            # 将停用词用列表的形式生成，之后查找停用词时会比较快
            self.stopWordDict = dict(zip(stopWordList, list(range(len(stopWordList)))))
            
    def dataGen(self):
        '''
         初始化训练集和验证集
        
        :param :
        :return: 
            
        '''
        
        # 初始化停用词
        self._readStopWord(self._stopWordSource)
        
        # 初始化数据集
        reviews, labels = self._readData(self._dataSource)
        
        # 初始化词汇-索引映射表和词向量矩阵
        self._genVocabulary(reviews)
        
        # 初始化训练集和测试集
        trainReviews, trainLabels, evalReviews, evalLabels = self._genTrainEvalData(reviews, labels, self._rate)
        self.trainReviews = trainReviews
        self.trainLabels = trainLabels
        
        self.evalReviews = evalReviews
        self.evalLabels = evalLabels
        
        
data = Dataset(config)
data.dataGen()

youre不存在于词向量中
youll不存在于词向量中
theyre不存在于词向量中
youve不存在于词向量中
werent不存在于词向量中
youd不存在于词向量中
hasnt不存在于词向量中
shouldnt不存在于词向量中
weve不存在于词向量中
theyve不存在于词向量中
1010不存在于词向量中
wouldve不存在于词向量中
hed不存在于词向量中
andor不存在于词向量中
couldve不存在于词向量中
810不存在于词向量中
itthe不存在于词向量中
710不存在于词向量中
theyd不存在于词向量中
writerdirector不存在于词向量中
moviei不存在于词向量中
iti不存在于词向量中
theyll不存在于词向量中
310不存在于词向量中
410不存在于词向量中
910不存在于词向量中
itll不存在于词向量中
lees不存在于词向量中
familys不存在于词向量中
disneys不存在于词向量中
filmi不存在于词向量中
210不存在于词向量中
shouldve不存在于词向量中
*12不存在于词向量中
shakespeares不存在于词向量中
hitlers不存在于词向量中
brazil:不存在于词向量中
freddys不存在于词向量中
fords不存在于词向量中
storys不存在于词向量中
ohara不存在于词向量中
stewarts不存在于词向量中
kellys不存在于词向量中
scotts不存在于词向量中
itthis不存在于词向量中
tonys不存在于词向量中
keatons不存在于词向量中
rosemarys不存在于词向量中
ches不存在于词向量中
themthe不存在于词向量中
timethe不存在于词向量中
branaghs不存在于词向量中
mustve不存在于词向量中
kubricks不存在于词向量中
lynchs不存在于词向量中
hitchcocks不存在于词向量中
bakshis不存在于词向量中
whove不存在于词向量中
allthe不存在于词向量中
whod不存在于词向量中
smiths不存在于词向量中
itit不存在于词向量中
obrien不存在于词向量中
cravens不存在于词向量中
timei不存在于词向量中
movieit不存在于词向量中
palmas不存在于词向量中
himthe不

shotthe不存在于词向量中
zanes不存在于词向量中
$399不存在于词向量中
sicas不存在于词向量中
releasedthe不存在于词向量中
notthis不存在于词向量中
geres不存在于词向量中
conaway不存在于词向量中
kopins不存在于词向量中
oooh不存在于词向量中
itselfthe不存在于词向量中
dosent不存在于词向量中
abouti不存在于词向量中
heartthe不存在于词向量中
clooneys不存在于词向量中
announcers不存在于词向量中
185:1不存在于词向量中
aka:不存在于词向量中
todos不存在于词向量中
dannings不存在于词向量中
livesthe不存在于词向量中
lisas不存在于词向量中
overi不存在于词向量中
weirs不存在于词向量中
grot不存在于词向量中
afis不存在于词向量中
mt不存在于词向量中
lawton不存在于词向量中
jellybean不存在于词向量中
kornman不存在于词向量中
genies不存在于词向量中
joness不存在于词向量中
afflecks不存在于词向量中
besti不存在于词向量中
paines不存在于词向量中
meit不存在于词向量中
becks不存在于词向量中
lengle不存在于词向量中
uhm不存在于词向量中
offthis不存在于词向量中
kusturicas不存在于词向量中
quaids不存在于词向量中
klines不存在于词向量中
himselfthe不存在于词向量中
eds不存在于词向量中
macleans不存在于词向量中
20000不存在于词向量中
grasshopper不存在于词向量中
movieeven不存在于词向量中
timeit不存在于词向量中
wasthis不存在于词向量中
longthe不存在于词向量中
phlox不存在于词向量中
timesthis不存在于词向量中
drakes不存在于词向量中
noras不存在于词向量中
mols不存在于词向量中
ashura不存在于词向量中
hannas不存在于词向量中
bogdanovichs不存在于词向量中
goshas不存在于词向量中
normans不存在于词向量中
themas不存在于词向量中
predictablethe不存在于词向量中
jindabyne

timeso不存在于词向量中
foleys不存在于词向量中
dramathe不存在于词向量中
1010!不存在于词向量中
thunderbolt不存在于词向量中
ladykillers不存在于词向量中
ofi不存在于词向量中
liviens不存在于词向量中
nolans不存在于词向量中
07不存在于词向量中
burgundians不存在于词向量中
jesters不存在于词向量中
facei不存在于词向量中
iton不存在于词向量中
bernsens不存在于词向量中
extinguishers不存在于词向量中
seenthis不存在于词向量中
lighti不存在于词向量中
darkheart不存在于词向量中
beforethis不存在于词向量中
chriss不存在于词向量中
anodyne不存在于词向量中
steeles不存在于词向量中
pasadena不存在于词向量中
darc不存在于词向量中
wiseman不存在于词向量中
scoops不存在于词向量中
watermans不存在于词向量中
facilitator不存在于词向量中
roomthe不存在于词向量中
alexandres不存在于词向量中
raina不存在于词向量中
nazgul不存在于词向量中
fancier不存在于词向量中
waya不存在于词向量中
filmto不存在于词向量中
inhe不存在于词向量中
conchata不存在于词向量中
twos不存在于词向量中
arisen不存在于词向量中
top-quality不存在于词向量中
fidos不存在于词向量中
happenedi不存在于词向量中
lifeas不存在于词向量中
dornwinkle不存在于词向量中
http:wwwpetitiononlinecomgh1215petitionhtml不存在于词向量中
tysons不存在于词向量中
713不存在于词向量中
showsthe不存在于词向量中
jehovahs不存在于词向量中
pias不存在于词向量中
yoshinaga不存在于词向量中
mauricio不存在于词向量中
narcolepsy不存在于词向量中
missi不存在于词向量中
directorthis不存在于词向量中
bards不存在于词向量中
yeari不存在于词向量中
zimmers不存在于词向量中
meds不存在于词向量中
aaro

In [4]:
print("train data shape: {}".format(data.trainReviews.shape))
print("train label shape: {}".format(data.trainLabels.shape))
print("eval data shape: {}".format(data.evalReviews.shape))

train data shape: (20000, 200)
train label shape: (20000, 1)
eval data shape: (5000, 200)


## 生成batch数据集

　　采用生成器的形式向模型输入batch数据集，（生成器可以避免将所有的数据加入到内存中）

In [5]:
# 输出batch数据集

def nextBatch(x, y, batchSize):
        '''
         生成batch数据集，用生成器的方式输出
        
        :param x:      list    
        :param y:      list    
        :param batchSize:      int    
        :return: 
            batchX        nnumpy array
            batchY        nnumpy array
            
        '''
    
        perm = np.arange(len(x))
        np.random.shuffle(perm)
        x = x[perm]
        y = y[perm]
        
        numBatches = len(x) // batchSize

        for i in range(numBatches):
            start = i * batchSize
            end = start + batchSize
            batchX = np.array(x[start: end], dtype="int64")
            batchY = np.array(y[start: end], dtype="float32")
            
            yield batchX, batchY

## Bi-LSTM + Attention模型

In [6]:
# 构建模型
class BiLSTMAttention(object):
    """
    Text CNN 用于文本分类
    """
    def __init__(self, config, wordEmbedding):

        # 定义模型的输入
        self.inputX = tf.placeholder(tf.int32, [None, config.sequenceLength], name="inputX")
        self.inputY = tf.placeholder(tf.float32, [None, 1], name="inputY")
        
        self.dropoutKeepProb = tf.placeholder(tf.float32, name="dropoutKeepProb")
        
        # 定义l2损失
        l2Loss = tf.constant(0.0)
        
        # 词嵌入层
        with tf.name_scope("embedding"):

            # 利用预训练的词向量初始化词嵌入矩阵
            self.W = tf.Variable(tf.cast(wordEmbedding, dtype=tf.float32, name="word2vec") ,name="W")
            # 利用词嵌入矩阵将输入的数据中的词转换成词向量，维度[batch_size, sequence_length, embedding_size]
            self.embeddedWords = tf.nn.embedding_lookup(self.W, self.inputX)
            
        # 定义两层双向LSTM的模型结构
        with tf.name_scope("Bi-LSTM"):
            for idx, hiddenSize in enumerate(config.model.hiddenSizes):
                with tf.name_scope("Bi-LSTM" + str(idx)):
                    # 定义前向LSTM结构
                    lstmFwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                                                                 output_keep_prob=self.dropoutKeepProb)
                    # 定义反向LSTM结构
                    lstmBwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                                                                 output_keep_prob=self.dropoutKeepProb)


                    # 采用动态rnn，可以动态的输入序列的长度，若没有输入，则取序列的全长
                    # outputs是一个元祖(output_fw, output_bw)，其中两个元素的维度都是[batch_size, max_time, hidden_size],fw和bw的hidden_size一样
                    # self.current_state 是最终的状态，二元组(state_fw, state_bw)，state_fw=[batch_size, s]，s是一个元祖(h, c)
                    outputs_, self.current_state = tf.nn.bidirectional_dynamic_rnn(lstmFwCell, lstmBwCell, 
                                                                                  self.embeddedWords, dtype=tf.float32,
                                                                                  scope="bi-lstm" + str(idx))
        
                    # 对outputs中的fw和bw的结果拼接 [batch_size, time_step, hidden_size * 2], 传入到下一层Bi-LSTM中
                    self.embeddedWords = tf.concat(outputs_, 2)
                
        # 将最后一层Bi-LSTM输出的结果分割成前向和后向的输出
        outputs = tf.split(self.embeddedWords, 2, -1)
        
        # 在Bi-LSTM+Attention的论文中，将前向和后向的输出相加
        with tf.name_scope("Attention"):
            H = outputs[0] + outputs[1]

            # 得到Attention的输出
            output = self.attention(H)
            outputSize = config.model.hiddenSizes[-1]
        
        # 全连接层的输出
        with tf.name_scope("output"):
            outputW = tf.get_variable(
                "outputW",
                shape=[outputSize, 1],
                initializer=tf.contrib.layers.xavier_initializer())
            
            outputB= tf.Variable(tf.constant(0.1, shape=[1]), name="outputB")
            l2Loss += tf.nn.l2_loss(outputW)
            l2Loss += tf.nn.l2_loss(outputB)
            self.predictions = tf.nn.xw_plus_b(output, outputW, outputB, name="predictions")
            self.binaryPreds = tf.cast(tf.greater_equal(self.predictions, 0.5), tf.float32, name="binaryPreds")
        
        # 计算二元交叉熵损失
        with tf.name_scope("loss"):
            
            losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.predictions, labels=self.inputY)
            self.loss = tf.reduce_mean(losses) + config.model.l2RegLambda * l2Loss
    
    def attention(self, H):
        """
        利用Attention机制得到句子的向量表示
        """
        # 获得最后一层LSTM的神经元数量
        hiddenSize = config.model.hiddenSizes[-1]
        
        # 初始化一个权重向量，是可训练的参数
        W = tf.Variable(tf.random_normal([hiddenSize], stddev=0.1))
        
        # 对Bi-LSTM的输出用激活函数做非线性转换
        M = tf.tanh(H)
        
        # 对W和M做矩阵运算，W=[batch_size, time_step, hidden_size]，计算前做维度转换成[batch_size * time_step, hidden_size]
        # newM = [batch_size, time_step, 1]，每一个时间步的输出由向量转换成一个数字
        newM = tf.matmul(tf.reshape(M, [-1, hiddenSize]), tf.reshape(W, [-1, 1]))
        
        # 对newM做维度转换成[batch_size, time_step]
        restoreM = tf.reshape(newM, [-1, config.sequenceLength])
        
        # 用softmax做归一化处理[batch_size, time_step]
        self.alpha = tf.nn.softmax(restoreM)
        
        # 利用求得的alpha的值对H进行加权求和，用矩阵运算直接操作
        r = tf.matmul(tf.transpose(H, [0, 2, 1]), tf.reshape(self.alpha, [-1, config.sequenceLength, 1]))
        
        # 将三维压缩成二维sequeezeR=[batch_size, hidden_size]
        sequeezeR = tf.squeeze(r)
        
        sentenceRepren = tf.tanh(sequeezeR)
        
        # 对Attention的输出可以做dropout处理
        output = tf.nn.dropout(sentenceRepren, self.dropoutKeepProb)
        
        return output

In [7]:
# 定义性能指标函数

def mean(item):
    return sum(item) / len(item)


def genMetrics(trueY, predY, binaryPredY):
    """
    生成acc和auc值
    """
    auc = roc_auc_score(trueY, predY)
    accuracy = accuracy_score(trueY, binaryPredY)
    precision = precision_score(trueY, binaryPredY)
    recall = recall_score(trueY, binaryPredY)
    
    return round(accuracy, 4), round(auc, 4), round(precision, 4), round(recall, 4)

In [8]:
# 训练模型

# 生成训练集和验证集
trainReviews = data.trainReviews
trainLabels = data.trainLabels
evalReviews = data.evalReviews
evalLabels = data.evalLabels

wordEmbedding = data.wordEmbedding

# 定义计算图
with tf.Graph().as_default():

    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    session_conf.gpu_options.allow_growth=True
    session_conf.gpu_options.per_process_gpu_memory_fraction = 0.9  # 配置gpu占用率  

    sess = tf.Session(config=session_conf)
    
    # 定义会话
    with sess.as_default():
        lstm = BiLSTMAttention(config, wordEmbedding)
        
        globalStep = tf.Variable(0, name="globalStep", trainable=False)
        # 定义优化函数，传入学习速率参数
        optimizer = tf.train.AdamOptimizer(config.training.learningRate)
        # 计算梯度,得到梯度和变量
        gradsAndVars = optimizer.compute_gradients(lstm.loss)
        # 将梯度应用到变量下，生成训练器
        trainOp = optimizer.apply_gradients(gradsAndVars, global_step=globalStep)
        
        # 用summary绘制tensorBoard
        gradSummaries = []
        for g, v in gradsAndVars:
            if g is not None:
                tf.summary.histogram("{}/grad/hist".format(v.name), g)
                tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
        
        outDir = os.path.abspath(os.path.join(os.path.curdir, "summarys"))
        print("Writing to {}\n".format(outDir))
        
        lossSummary = tf.summary.scalar("loss", lstm.loss)
        summaryOp = tf.summary.merge_all()
        
        trainSummaryDir = os.path.join(outDir, "train")
        trainSummaryWriter = tf.summary.FileWriter(trainSummaryDir, sess.graph)
        
        evalSummaryDir = os.path.join(outDir, "eval")
        evalSummaryWriter = tf.summary.FileWriter(evalSummaryDir, sess.graph)
        
        
        # 初始化所有变量
#         saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)
        
        # 保存模型的一种方式，保存为pb文件
#         builder = tf.saved_model.builder.SavedModelBuilder("../model/Bi-LSTM/savedModel")
        sess.run(tf.global_variables_initializer())

        def trainStep(batchX, batchY):
            """
            训练函数
            """   
            feed_dict = {
              lstm.inputX: batchX,
              lstm.inputY: batchY,
              lstm.dropoutKeepProb: config.model.dropoutKeepProb
            }
            _, summary, step, loss, predictions, binaryPreds = sess.run(
                [trainOp, summaryOp, globalStep, lstm.loss, lstm.predictions, lstm.binaryPreds],
                feed_dict)
            timeStr = datetime.datetime.now().isoformat()
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(timeStr, step, loss, acc, auc, precision, recall))
            trainSummaryWriter.add_summary(summary, step)

        def devStep(batchX, batchY):
            """
            验证函数
            """
            feed_dict = {
              lstm.inputX: batchX,
              lstm.inputY: batchY,
              lstm.dropoutKeepProb: 1.0
            }
            summary, step, loss, predictions, binaryPreds = sess.run(
                [summaryOp, globalStep, lstm.loss, lstm.predictions, lstm.binaryPreds],
                feed_dict)
            
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            
            evalSummaryWriter.add_summary(summary, step)
            
            return loss, acc, auc, precision, recall
        
        for i in range(config.training.epoches):
            # 训练模型
            print("start training model")
            for batchTrain in nextBatch(trainReviews, trainLabels, config.batchSize):
                trainStep(batchTrain[0], batchTrain[1])

                currentStep = tf.train.global_step(sess, globalStep) 
                if currentStep % config.training.evaluateEvery == 0:
                    print("\nEvaluation:")
                    
                    losses = []
                    accs = []
                    aucs = []
                    precisions = []
                    recalls = []
                    
                    for batchEval in nextBatch(evalReviews, evalLabels, config.batchSize):
                        loss, acc, auc, precision, recall = devStep(batchEval[0], batchEval[1])
                        losses.append(loss)
                        accs.append(acc)
                        aucs.append(auc)
                        precisions.append(precision)
                        recalls.append(recall)
                        
                    time_str = datetime.datetime.now().isoformat()
                    print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(time_str, currentStep, mean(losses), 
                                                                                                       mean(accs), mean(aucs), mean(precisions),
                                                                                                       mean(recalls)))
                    
#                 if currentStep % config.training.checkpointEvery == 0:
#                     # 保存模型的另一种方法，保存checkpoint文件
#                     path = saver.save(sess, "../model/Bi-LSTM/model/my-model", global_step=currentStep)
#                     print("Saved model checkpoint to {}\n".format(path))
                    
#         inputs = {"inputX": tf.saved_model.utils.build_tensor_info(lstm.inputX),
#                   "keepProb": tf.saved_model.utils.build_tensor_info(lstm.dropoutKeepProb)}

#         outputs = {"binaryPreds": tf.saved_model.utils.build_tensor_info(lstm.binaryPreds)}

#         prediction_signature = tf.saved_model.signature_def_utils.build_signature_def(inputs=inputs, outputs=outputs,
#                                                                                       method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)
#         legacy_init_op = tf.group(tf.tables_initializer(), name="legacy_init_op")
#         builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING],
#                                             signature_def_map={"predict": prediction_signature}, legacy_init_op=legacy_init_op)

#         builder.save()

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name bi-lstm0/fw/lstm_cell/kernel:0/grad/hist is illegal; using bi-lstm0/fw/lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name bi-lstm0/fw/lstm_cell/kernel:0/grad/sparsity is illegal; using bi-lstm0/fw/lstm_cell/kernel_0/grad/sparsity instead.
INFO:tensorflow:Summary name bi-lstm0/fw/lstm_cell/bias:0/grad/hist is illegal; using bi-lstm0/fw/lstm_cell/bias_0/grad/hist instead.
INFO:tensorflow:Summary name bi-lstm0/fw/lstm_cell/bias:0/grad/sparsity is illegal; using bi-lstm0/fw/lstm_cell/bias_0/grad/sparsity instead.
INFO:tensorflow:Summary name bi-lstm0/bw/lstm_cell/kernel:0/grad/hist is illegal; using bi-lstm0/bw/lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name bi-lstm0/bw/lstm_cell/kernel:0/grad/sparsity is illegal; 

2019-03-18T10:32:15.035090, step: 43, loss: 0.4230799078941345, acc: 0.8125, auc: 0.9042, precision: 0.8485, recall: 0.8
2019-03-18T10:32:22.578345, step: 44, loss: 0.44611892104148865, acc: 0.7656, auc: 0.878, precision: 0.8889, recall: 0.6667
2019-03-18T10:32:31.756723, step: 45, loss: 0.4243914783000946, acc: 0.7969, auc: 0.8902, precision: 0.8864, recall: 0.65
2019-03-18T10:32:40.238237, step: 46, loss: 0.33859604597091675, acc: 0.8281, auc: 0.9348, precision: 0.8596, recall: 0.7778
2019-03-18T10:32:48.246060, step: 47, loss: 0.33185142278671265, acc: 0.8125, auc: 0.9361, precision: 0.8302, recall: 0.7458
2019-03-18T10:32:55.702024, step: 48, loss: 0.29110267758369446, acc: 0.8906, auc: 0.9504, precision: 0.9403, recall: 0.863
2019-03-18T10:33:03.701251, step: 49, loss: 0.47441259026527405, acc: 0.7891, auc: 0.8812, precision: 0.7536, recall: 0.8387
2019-03-18T10:33:12.230809, step: 50, loss: 0.4076458513736725, acc: 0.8359, auc: 0.9082, precision: 0.8772, recall: 0.7812
2019-03-18

2019-03-18T10:47:19.004482, step: 108, loss: 0.28726932406425476, acc: 0.8984, auc: 0.9586, precision: 0.8814, recall: 0.8966
2019-03-18T10:47:29.061152, step: 109, loss: 0.2769928574562073, acc: 0.8984, auc: 0.9558, precision: 0.9492, recall: 0.8485
2019-03-18T10:47:38.535152, step: 110, loss: 0.33353573083877563, acc: 0.8438, auc: 0.9299, precision: 0.8772, recall: 0.7937
2019-03-18T10:47:48.678463, step: 111, loss: 0.2757161855697632, acc: 0.8828, auc: 0.9587, precision: 0.8983, recall: 0.8548
2019-03-18T10:47:58.512624, step: 112, loss: 0.48012638092041016, acc: 0.7734, auc: 0.8742, precision: 0.9167, recall: 0.6377
2019-03-18T10:48:08.209312, step: 113, loss: 0.42112278938293457, acc: 0.7891, auc: 0.8914, precision: 0.8478, recall: 0.661
2019-03-18T10:48:18.367769, step: 114, loss: 0.311423122882843, acc: 0.8672, auc: 0.9414, precision: 0.8966, recall: 0.8254
2019-03-18T10:48:27.853957, step: 115, loss: 0.25307148694992065, acc: 0.8906, auc: 0.9682, precision: 0.9516, recall: 0.84

2019-03-18T10:57:40.612847, step: 174, loss: 0.21732580661773682, acc: 0.8906, auc: 0.9726, precision: 0.9385, recall: 0.8592
2019-03-18T10:57:49.550097, step: 175, loss: 0.19443362951278687, acc: 0.8984, auc: 0.9807, precision: 0.931, recall: 0.8571
2019-03-18T10:57:58.643249, step: 176, loss: 0.21213006973266602, acc: 0.8828, auc: 0.9725, precision: 0.9219, recall: 0.8551
2019-03-18T10:58:07.076521, step: 177, loss: 0.23302946984767914, acc: 0.8984, auc: 0.9741, precision: 0.9492, recall: 0.8485
2019-03-18T10:58:16.369095, step: 178, loss: 0.3349497318267822, acc: 0.8828, auc: 0.9397, precision: 0.8889, recall: 0.875
2019-03-18T10:58:25.213968, step: 179, loss: 0.21833807229995728, acc: 0.9141, auc: 0.9796, precision: 0.8519, recall: 0.9388
2019-03-18T10:58:34.060719, step: 180, loss: 0.17783239483833313, acc: 0.9297, auc: 0.981, precision: 0.9552, recall: 0.9143
2019-03-18T10:58:42.999974, step: 181, loss: 0.21955403685569763, acc: 0.8828, auc: 0.9744, precision: 0.9464, recall: 0.8

2019-03-18T11:13:44.709987, step: 239, loss: 0.2520262598991394, acc: 0.875, auc: 0.9745, precision: 1.0, recall: 0.7778
2019-03-18T11:13:54.445147, step: 240, loss: 0.3578265309333801, acc: 0.8438, auc: 0.9748, precision: 1.0, recall: 0.697
2019-03-18T11:14:03.850921, step: 241, loss: 0.31081825494766235, acc: 0.7969, auc: 0.9993, precision: 1.0, recall: 0.6119
2019-03-18T11:14:12.981403, step: 242, loss: 0.24328619241714478, acc: 0.8906, auc: 0.9759, precision: 0.9706, recall: 0.8462
2019-03-18T11:14:22.297933, step: 243, loss: 0.24903614819049835, acc: 0.8906, auc: 0.9718, precision: 0.8649, recall: 0.9412
2019-03-18T11:14:31.694393, step: 244, loss: 0.3693750202655792, acc: 0.8594, auc: 0.9731, precision: 0.8, recall: 0.9697
2019-03-18T11:14:41.115210, step: 245, loss: 0.4397905170917511, acc: 0.8281, auc: 0.9696, precision: 0.7317, recall: 1.0
2019-03-18T11:14:49.762077, step: 246, loss: 0.3055890202522278, acc: 0.8906, auc: 0.9653, precision: 0.8442, recall: 0.9701
2019-03-18T11:

2019-03-18T11:29:54.258600, step: 304, loss: 0.21646299958229065, acc: 0.9062, auc: 0.9783, precision: 0.9692, recall: 0.863
2019-03-18T11:30:03.952297, step: 305, loss: 0.1878250688314438, acc: 0.9375, auc: 0.9763, precision: 0.9344, recall: 0.9344
2019-03-18T11:30:13.353215, step: 306, loss: 0.2547404170036316, acc: 0.9219, auc: 0.9573, precision: 0.9344, recall: 0.9048
2019-03-18T11:30:22.614968, step: 307, loss: 0.27211838960647583, acc: 0.8828, auc: 0.9595, precision: 0.9074, recall: 0.8305
2019-03-18T11:30:32.096796, step: 308, loss: 0.2222478687763214, acc: 0.9062, auc: 0.9722, precision: 0.9123, recall: 0.8814
2019-03-18T11:30:43.051051, step: 309, loss: 0.16631051898002625, acc: 0.9219, auc: 0.9853, precision: 0.9143, recall: 0.9412
2019-03-18T11:30:53.584619, step: 310, loss: 0.15369680523872375, acc: 0.9453, auc: 0.9858, precision: 0.9841, recall: 0.9118
2019-03-18T11:31:03.056080, step: 311, loss: 0.20419207215309143, acc: 0.9297, auc: 0.9739, precision: 0.9492, recall: 0.9

2019-03-18T11:41:03.756324, step: 370, loss: 0.11895917356014252, acc: 0.9609, auc: 0.9933, precision: 1.0, recall: 0.9286
2019-03-18T11:41:13.643769, step: 371, loss: 0.17250284552574158, acc: 0.9531, auc: 0.9793, precision: 0.9623, recall: 0.9273
2019-03-18T11:41:23.289518, step: 372, loss: 0.12436855584383011, acc: 0.9609, auc: 0.9927, precision: 0.9844, recall: 0.9403
2019-03-18T11:41:32.773150, step: 373, loss: 0.17717881500720978, acc: 0.9141, auc: 0.9795, precision: 0.9104, recall: 0.9242
2019-03-18T11:41:42.141805, step: 374, loss: 0.13250041007995605, acc: 0.9531, auc: 0.9954, precision: 0.9118, recall: 1.0
2019-03-18T11:41:52.255988, step: 375, loss: 0.11966954171657562, acc: 0.9531, auc: 0.9933, precision: 0.9444, recall: 0.9714
2019-03-18T11:42:01.898936, step: 376, loss: 0.17106366157531738, acc: 0.9219, auc: 0.9856, precision: 0.9474, recall: 0.8852
2019-03-18T11:42:11.466114, step: 377, loss: 0.07673950493335724, acc: 0.9688, auc: 0.9972, precision: 0.9861, recall: 0.959

2019-03-18T12:00:31.815777, step: 435, loss: 0.2460184097290039, acc: 0.9141, auc: 0.9836, precision: 1.0, recall: 0.8226
2019-03-18T12:00:43.736392, step: 436, loss: 0.1985611915588379, acc: 0.9297, auc: 0.9737, precision: 0.9437, recall: 0.9306
2019-03-18T12:00:56.761218, step: 437, loss: 0.1280064731836319, acc: 0.9531, auc: 0.9907, precision: 0.9595, recall: 0.9595
2019-03-18T12:01:09.233358, step: 438, loss: 0.1976977288722992, acc: 0.9375, auc: 0.987, precision: 0.8939, recall: 0.9833
2019-03-18T12:01:21.422820, step: 439, loss: 0.1408531665802002, acc: 0.9688, auc: 0.9847, precision: 0.9733, recall: 0.9733
2019-03-18T12:01:34.396082, step: 440, loss: 0.1499888002872467, acc: 0.9453, auc: 0.9907, precision: 0.9333, recall: 0.9492
2019-03-18T12:01:46.673310, step: 441, loss: 0.1437181830406189, acc: 0.9297, auc: 0.9875, precision: 0.9492, recall: 0.9032
2019-03-18T12:01:58.929851, step: 442, loss: 0.04027952998876572, acc: 0.9844, auc: 1.0, precision: 1.0, recall: 0.9623
2019-03-1

2019-03-18T12:19:42.316597, step: 501, loss: 0.06200036033987999, acc: 0.9688, auc: 0.999, precision: 0.9531, recall: 0.9839
2019-03-18T12:19:52.687189, step: 502, loss: 0.11317858099937439, acc: 0.9688, auc: 0.9993, precision: 0.9394, recall: 1.0
2019-03-18T12:20:02.329690, step: 503, loss: 0.08474566042423248, acc: 0.9688, auc: 0.998, precision: 0.9492, recall: 0.9825
2019-03-18T12:20:11.805676, step: 504, loss: 0.05391063168644905, acc: 0.9844, auc: 0.998, precision: 0.9841, recall: 0.9841
2019-03-18T12:20:21.846370, step: 505, loss: 0.146020770072937, acc: 0.9609, auc: 0.9956, precision: 1.0, recall: 0.9167
2019-03-18T12:20:32.336439, step: 506, loss: 0.155074805021286, acc: 0.9375, auc: 0.9875, precision: 0.9851, recall: 0.9041
2019-03-18T12:20:42.860592, step: 507, loss: 0.1370777189731598, acc: 0.9609, auc: 0.984, precision: 0.971, recall: 0.9571
2019-03-18T12:20:53.438516, step: 508, loss: 0.09789043664932251, acc: 0.9609, auc: 0.9975, precision: 0.9444, recall: 0.9855
2019-03-

2019-03-18T12:30:44.761082, step: 568, loss: 0.06160542741417885, acc: 0.9844, auc: 0.9995, precision: 1.0, recall: 0.9672
2019-03-18T12:30:54.051941, step: 569, loss: 0.08530521392822266, acc: 0.9688, auc: 0.9951, precision: 0.9839, recall: 0.9531
2019-03-18T12:31:03.391703, step: 570, loss: 0.1510189175605774, acc: 0.9219, auc: 0.9892, precision: 0.9636, recall: 0.8689
2019-03-18T12:31:13.036322, step: 571, loss: 0.10333894938230515, acc: 0.9609, auc: 0.9961, precision: 0.9545, recall: 0.9692
2019-03-18T12:31:21.668115, step: 572, loss: 0.056474581360816956, acc: 0.9844, auc: 0.9995, precision: 0.9683, recall: 1.0
2019-03-18T12:31:31.101226, step: 573, loss: 0.11123862862586975, acc: 0.9766, auc: 0.9973, precision: 0.9559, recall: 1.0
2019-03-18T12:31:40.445394, step: 574, loss: 0.06915938854217529, acc: 0.9766, auc: 0.997, precision: 0.9722, recall: 0.9859
2019-03-18T12:31:49.835556, step: 575, loss: 0.05532423034310341, acc: 0.9844, auc: 0.998, precision: 1.0, recall: 0.963
2019-03